In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
# %load_ext memory_profile
import numpy as np
import pandas as pd
import time
import copy
import pickle

In [2]:
import diem.polarize as pol
import diem.tests as tests
from diem.diemtype import DiemType, import_raw_test_data, new_import_raw_test_data
from diem.diemtype import save_DiemType, load_DiemType

In [3]:
from diem.contigs import Contig, Interval, get_intervals, build_contig_matrix

In [4]:
from diem.utils import plot_painting
from diem.diemtype import flip_polarity

## quick checks of i) Data import, ii) Polarization on test initialization iii) thresholding of that dataset and iii) smoothing of thresholded data

**check data importing**

The test data importing is all defined, file by file, in the 'import_raw_test_data()' function

In [73]:
#### CAREFUL! uncomment the following and it will over-write the current 'solutions' for testing.
#### Do this ONLY when (i) the DiemType class name has been changed or (ii) an attribute NAME has changed, or additional attributes are needed in the test.
#### for (ii), the test data is stored at dictionaries, so an 'updated' DiemType with additional attributes can still import 'older' versions
#### and the attributes are strictly defined so as longas both test objects have those attributes the test functions will work.  

# dInputSolution = new_import_raw_test_data()
# dPolarizedSolution = dInputSolution.polarize(boolTestData=True)
# dSortedSolution = dPolarizedSolution.copy()
# dSortedSolution.sort()
# dThresholdedSolution = dSortedSolution.apply_threshold(-20)
# dSmoothedSolution = dThresholdedSolution.smooth(1e-4)
# dContigSolution = dSmoothedSolution.copy()
# dContigSolution.create_contig_matrix()

# save_DiemType(dInputSolution,'../tests/test_data/input_solution.DiemType.dict.pkl')
# save_DiemType(dPolarizedSolution,'../tests/test_data/polarized_solution.DiemType.dict.pkl')
# save_DiemType(dSortedSolution,'../tests/test_data/sorted_solution.DiemType.dict.pkl')
# save_DiemType(dThresholdedSolution,'../tests/test_data/thresholded_solution.DiemType.dict.pkl')
# save_DiemType(dSmoothedSolution,'../tests/test_data/smoothed_solution.DiemType.dict.pkl')
# save_DiemType(dContigSolution, '../tests/test_data/contig_solution.DiemType.dict.pkl')

In [5]:

#### importing the DiemType instances used for testing modifications to the code
dInputSolution = load_DiemType('../tests/test_data/input_solution.DiemType.dict.pkl')
dPolarizedSolution = load_DiemType('../tests/test_data/polarized_solution.DiemType.dict.pkl')
dSortedSolution = load_DiemType('../tests/test_data/sorted_solution.DiemType.dict.pkl')
dThresholdedSolution = load_DiemType('../tests/test_data/thresholded_solution.DiemType.dict.pkl')
dSmoothedSolution = load_DiemType('../tests/test_data/smoothed_solution.DiemType.dict.pkl')
dContigSolution = load_DiemType('../tests/test_data/contig_solution.DiemType.dict.pkl')

# plot_painting(dInputSolution.DMBC[0])
# plot_painting(dPolarizedSolution.DMBC[0])
# plot_painting(dSortedSolution.DMBC[0])
# plot_painting(dThresholdedSolution.DMBC[0])
# plot_painting(dSmoothedSolution.DMBC[0])

**checking input of data from 'new format' files**

In [9]:
dInputTestNewFormatData = new_import_raw_test_data()

In [10]:
tests.compare_DiemTypes(dInputTestNewFormatData,dInputTestNewFormatData)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
now checking contigMatrix attribute
All tests passed!


**checking polarization of the *correctly* imported or read-in diplotypeobject**

In [11]:
dPolarizedTest = dInputSolution.polarize(boolTestData=True) #note that I use the dInputSOLUTION so that we are sure the tests are run on the right data

convert state matrix to Marray
initializing test polarity
running EM algorithm on a single core using diem_linear
starting
 num individuals in MBC is  14
iteration  0   num 0 polarity =  453158   num 1 polarity =  453157
time for iteration in seconds:  3.949525833129883
iteration  1   num 0 polarity =  585141   num 1 polarity =  321174
time for iteration in seconds:  3.4656484127044678
iteration  2   num 0 polarity =  628702   num 1 polarity =  277613
time for iteration in seconds:  3.5893170833587646
iteration  3   num 0 polarity =  630008   num 1 polarity =  276307
time for iteration in seconds:  3.4902594089508057
iteration  4   num 0 polarity =  630170   num 1 polarity =  276145
time for iteration in seconds:  3.8516223430633545
iteration  5   num 0 polarity =  630196   num 1 polarity =  276119
time for iteration in seconds:  3.6648755073547363
iteration  6   num 0 polarity =  630193   num 1 polarity =  276122
polarity found
time for iteration in seconds:  3.6563847064971924
polari

In [12]:
tests.compare_DiemTypes(dPolarizedTest,dPolarizedSolution)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Attribute  initialPolByChr  differs (one is None, the other is not)
Now checking non-list attributes
Attribute  HIs  differs (one is None, the other is not)
Now checking float attributes
now checking individual exclusions
now checking site exclusions
previous attributes failed, so skipping contigMatrix check
The following attributes differed:  ['initialPolByChr', 'HIs']


In [13]:
dPolarizedTestParallel = dInputSolution.polarize(boolTestData=True,ncores=3) #note that I use the dInputSOLUTION so that we are sure the tests are run on the right data

convert state matrix to Marray
initializing test polarity
running EM algorithm in parallel using diem_parallel with  3  cores
starting
using  3  CPUs
iteration  0   num 0 polarity =  453158   num 1 polarity =  453157
time to flip:  1.4071629047393799
iteration  1   num 0 polarity =  585141   num 1 polarity =  321174
time to flip:  1.1684553623199463
iteration  2   num 0 polarity =  628702   num 1 polarity =  277613
time to flip:  1.2260680198669434
iteration  3   num 0 polarity =  630008   num 1 polarity =  276307
time to flip:  1.1465766429901123
iteration  4   num 0 polarity =  630170   num 1 polarity =  276145
time to flip:  1.4002537727355957
iteration  5   num 0 polarity =  630196   num 1 polarity =  276119
time to flip:  1.2174413204193115
iteration  6   num 0 polarity =  630193   num 1 polarity =  276122
time to flip:  1.2383661270141602
polarity found
polarity found
updating polarizations, DIs, Supports, initialPolarity,and state matrices


In [14]:
tests.compare_DiemTypes(dPolarizedTestParallel,dPolarizedSolution)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Attribute  initialPolByChr  differs (one is None, the other is not)
Now checking non-list attributes
Attribute  HIs  differs (one is None, the other is not)
Now checking float attributes
now checking individual exclusions
now checking site exclusions
previous attributes failed, so skipping contigMatrix check
The following attributes differed:  ['initialPolByChr', 'HIs']


**checking the sort function**

In [17]:
dSortedTest.chrPloidies

array([[2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]])

In [18]:
dSortedSolution.chrPloidies

array([[1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]])

In [16]:
dSortedTest = dPolarizedSolution.copy()
dSortedTest.sort()
tests.compare_DiemTypes(dSortedTest,dSortedSolution)

new HIs computer and individuals resorted by HI
Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
Attribute  chrPloidies  differs for chromosome  0
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
previous attributes failed, so skipping contigMatrix check
The following attributes differed:  ['chrPloidies']


**checking the thresholding of the *correctly* imported or read-in diplotype object**


In [22]:
dThresholdedTest = dSortedSolution.apply_threshold(-20)

proportion of each chromosome retained after thresholding:
[np.float64(0.49810460543643115), np.float64(0.4922365573263625)]
thresholding done
hybrid indices have likely changed, but have not been updated
to update HIs without re-sorting, update the attribute a.HIs = a.computeHIs()
You may also call the sort() method on the resulting data if you wish to both re-compute and resort


In [23]:
tests.compare_DiemTypes(dThresholdedTest,dThresholdedSolution)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
now checking contigMatrix attribute
All tests passed!


**checking smoothing function**

In [10]:
dSmoothedTest = dThresholdedSolution.smooth(1e-4)
tests.compare_DiemTypes(dSmoothedTest,dSmoothedSolution)

smoothing the data with scale  0.0001
using parallel smoothing
smoothing done, but not resorting by HI. You may call the sort() method on the resulting data if you wish to resort
Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
now checking contigMatrix attribute
All tests passed!


**checking the contig creationg function**

In [50]:
dContigTest = dSmoothedSolution.copy()
dContigTest.create_contig_matrix()

contig matrix created and stored in self.contigMatrix


In [51]:
tests.compare_DiemTypes(dContigTest,dContigSolution)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
now checking contigMatrix attribute
All tests passed!


# exporting contigs to bed file

In [53]:
outputDir = '/home/derek/git/diemPy/example_data/contig_output'
dContigSolution.intervals_to_bed(outputDir)

BED files created in directory:  /home/derek/git/diemPy/example_data/contig_output


In [54]:
from pympler import asizeof

In [55]:
size = asizeof.asizeof(dSmoothedSolution)
print(size/1024/1024)

size = asizeof.asizeof(dContigSolution)
print(size/1024/1024)

131.25335693359375
131.25357055664062


In [56]:
dTest = dSmoothedSolution.copy()
dTest.create_contig_matrix()

contig matrix created and stored in self.contigMatrix


In [60]:
dTest = dThresholdedSolution.copy()
dTest.create_contig_matrix()
size = asizeof.asizeof(dTest)
print(size/1024/1024)  # in MBb


contig matrix created and stored in self.contigMatrix
65.62639617919922


In [61]:
dTest = dSmoothedSolution.copy()
dTest.create_contig_matrix()
size = asizeof.asizeof(dTest)
print(size/1024/1024)  # in MB

contig matrix created and stored in self.contigMatrix
65.62641906738281


In [62]:
dTest = dPolarizedSolution.copy()
dTest.create_contig_matrix()
size = asizeof.asizeof(dTest)
print(size/1024/1024)  # in MB

contig matrix created and stored in self.contigMatrix
125.33002471923828


In [58]:
tests.compare_DiemTypes(dTest,dContigSolution)

Now checking list attributes
checking DMBC
checking PolByChr
checking DIByChr
checking SupportByChr
checking posByChr
checking MapBC
checking chrPloidies
checking initialPolByChr
Now checking non-list attributes
Now checking float attributes
now checking individual exclusions
now checking site exclusions
now checking contigMatrix attribute
All tests passed!


In [64]:
testMList = [pol.stateMatrix_to_MArray(x) for x in dInputSolution.DMBC]


In [65]:
size = asizeof.asizeof(testMList)
print(size/1024/1024)  # in MB

48.40254211425781


In [69]:
sum([np.size(x) for x in dInputSolution.DMBC])/1024/1024  # in MB

12.100610733032227

In [82]:
nSites = [np.size(x) for x in dInputSolution.DMBC]
np.sum(nSites)/1024/1024*100*10

np.float64(12100.610733032227)

In [77]:
nInds = len(dInputSolution.indNames)
nSitesByChr = np.array([len(x[0]) for x in dInputSolution.DMBC])
nSitesByChr

array([550545, 355770])

In [81]:
sum(nInds*nSitesByChr)

np.int64(12688410)